In [ ]:
import string
import pandas as pd
import tkinter as tk
from tkinter import scrolledtext
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

df = pd.read_excel("ai_faqs.xlsx")

questions = df["question"].tolist()
answers = df["answer"].tolist()
print(" Total AI FAQs loaded:", len(questions))

def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    tokens = text.split()
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

cleaned_questions = [clean_text(q) for q in questions]

vectorizer = TfidfVectorizer()
faq_vectors = vectorizer.fit_transform(cleaned_questions)
print("Text vectorization completed")

def chatbot_response(user_input):
    cleaned_input = clean_text(user_input)
    user_vector = vectorizer.transform([cleaned_input])
    similarity_scores = cosine_similarity(user_vector, faq_vectors)
    best_match_index = similarity_scores.argmax()
    confidence = similarity_scores[0][best_match_index]
    if confidence < 0.25:
        return "Sorry, I could not find a relevant answer. Please rephrase your question."
    else:
        return answers[best_match_index]

def send_message():
    user_text = user_entry.get()
    if user_text.strip() == "":
        return
    chat_area.configure(state='normal')
    chat_area.insert(tk.END, "You: " + user_text + "\n")
    response = chatbot_response(user_text)
    chat_area.insert(tk.END, "Bot: " + response + "\n\n")
    chat_area.yview(tk.END)
    user_entry.delete(0, tk.END)
    chat_area.configure(state='disabled')

root = tk.Tk()
root.title("AI FAQs Chatbot - Mahboob Ali")
root.geometry("700x500")

chat_area = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=80, height=25)
chat_area.pack(pady=10)
chat_area.configure(state='disabled')

user_entry = tk.Entry(root, width=60)
user_entry.pack(pady=5)
user_entry.focus()

send_button = tk.Button(root, text="Ask", command=send_message)
send_button.pack(pady=5)

root.bind('<Return>', lambda event: send_message())

root.mainloop()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mahboob\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


 Total AI FAQs loaded: 151
Text vectorization completed
